# HW5 - Binary Classification using Swin Transformer
Intelligent Analysis of Biomedical Images

Fall 2023

- Name:
- Student id:

**Introduction**:

In this educational notebook, we will explore binary classification on the BreastMNIST dataset, consisting of 780 breast ultrasound images. This task involves using the Swin Transformer, a cutting-edge neural network model, to distinguish between benign (including normal) and malignant cases.

We'll tackle the common challenge of class imbalance in medical datasets and learn to improve model performance using class weights. Additionally, we'll delve into evaluating our model with ROC curves and AUC, essential tools for assessing performance in medical image classification.


## Packages & Modules

In [ ]:
from IPython.display import clear_output

import torch
from tqdm import tqdm
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
import matplotlib.pyplot as plt
from collections import Counter

from torchmetrics import Accuracy, F1Score, ConfusionMatrix

!pip install torchmetrics
!pip install timm
!pip install medmnist

from torchmetrics import Accuracy
from timm import create_model
import medmnist

device = "cuda" if torch.cuda.is_available() else "cpu"
clear_output()

## Data

The BreastMNIST dataset is simplified into a binary classification problem, merging normal and benign images into a single class, contrasting against malignant images. The dataset, originally 1×500×500 in size, is resized to 1×28×28 and split into training, validation, and test sets. We address the class imbalance by applying class weights in our loss function, enhancing the focus on underrepresented classes.

In [ ]:
info = medmnist.INFO['breastmnist']
n_channels = info['n_channels']

DataClass = getattr(medmnist, info['python_class'])

In [ ]:
print(info['n_samples'])
print(info['label'])

{'train': 546, 'val': 78, 'test': 156}
{'0': 'malignant', '1': 'normal, benign'}


In [ ]:
# preprocessing
train_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.AugMix(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[.5], std=[.5])
])



BATCH_SIZE = 16

# load the data
train_dataset = DataClass(split='train', transform=train_transform, download=True)
val_dataset = DataClass(split='val', transform=train_transform, download=True)
test_dataset = DataClass(split='test', transform=test_transform, download=True)

# Create instances of your dataset and data loader
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
images, labels = next(iter(train_loader))
images = images[:16]
labels = labels[:16]

fig, axes = plt.subplots(2, 8, figsize=(8, 4))

for i, ax in enumerate(axes.flatten()):
    # Convert the tensor image to numpy array format
    image = images[i].numpy().transpose(1, 2, 0)
    # Normalize the image to the range [0, 1]
    image = (image + 1) / 2
    ax.imshow(image, cmap='gray')
    ax.axis("off")
    ax.set_title(f"{info['label'][str(labels[i].item())]}", fontsize=8)

plt.tight_layout()
plt.show()

### Train

In [ ]:
model = create_model(model_name="swin_tiny_patch4_window7_224",
                        pretrained=True,
                        num_classes=1,
                        in_chans=n_channels)
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
from torchmetrics import Accuracy, F1Score, ConfusionMatrix

def train_one_epoch(model, train_loader, loss_fn, optimizer, epoch=None):
    model.train()
    total_loss = 0.0
    total_acc = 0.0

    # Hint: Implement one training epoch using the provided model, data loader, loss function, and optimizer.
    # Your code should include forward and backward passes, updating the model's parameters, and tracking metrics.
    # Calculate and return the average loss, accuracy, F1-score, and confusion matrix for the epoch.
    # Your code here [20 score]

    return model, avg_loss, avg_acc, avg_f1, cm


In [ ]:
def validate_one_epoch(model, val_loader, criterion):
    model.eval()
    total_loss = 0.0
    total_acc = 0.0

    # Hint: Implement one validation epoch using the provided model, data loader, and loss criterion.
    # Calculate and return the average loss, accuracy, F1-score, and confusion matrix for the epoch.
    # Your code here [20 score]

    return avg_loss, avg_acc, avg_f1, cm

In [ ]:
# Training Loop
n_epochs = 100
loss_train_hist = []
loss_valid_hist = []
acc_train_hist = []
acc_valid_hist = []
f1_train_hist = []
f1_valid_hist = []
cm_train_hist = []
cm_valid_hist = []

for epoch in range(n_epochs):
    torch.cuda.empty_cache()

    model, loss_train, acc_train, f1_train, cm_train = train_one_epoch(model, train_loader, criterion, optimizer, epoch)
    loss_valid, acc_valid, f1_valid, cm_valid = validate_one_epoch(model, val_loader, criterion)

    # Append metrics to history
    loss_train_hist.append(loss_train)
    loss_valid_hist.append(loss_valid)
    acc_train_hist.append(acc_train)
    acc_valid_hist.append(acc_valid)
    f1_train_hist.append(f1_train)
    f1_valid_hist.append(f1_valid)
    cm_train_hist.append(cm_train)
    cm_valid_hist.append(cm_valid)

    print(f"Epoch: {epoch+1}, Train Loss: {loss_train:.2f}, Train Acc: {acc_train:.2f}, Train F1: {f1_train:.2f}, Valid Loss: {loss_valid:.2f}, Valid Acc: {acc_valid:.2f}, Valid F1: {f1_valid:.2f}")


In [ ]:
def plot_performance(loss_train_hist, loss_valid_hist, acc_train_hist, acc_valid_hist):
    plt.figure(figsize=(6, 3))

    plt.subplot(1, 2, 1)
    plt.plot(loss_train_hist, label='Train Loss')
    plt.plot(loss_valid_hist, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(acc_train_hist, label='Train Accuracy')
    plt.plot(acc_valid_hist, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()

In [ ]:
plot_performance(loss_train_hist, loss_valid_hist, acc_train_hist, acc_valid_hist)

**Discussing the Limitations of Accuracy**

While accuracy is a commonly used metric, it can be misleading, especially in cases of class imbalance. In such scenarios, a model might predict the majority class for all instances, resulting in high accuracy but poor model performance in practical terms.

**Why F1-Score?**
The F1-score is a more robust metric in these cases as it balances precision and recall, providing a better measure of the classifier's performance, especially when the classes are imbalanced.

**Importance of Confusion Matrix**
The confusion matrix provides an in-depth view of the classifier's performance. It shows not just the overall accuracy but how the model performs on each individual class, revealing any biases or weaknesses in the model's predictions.


In [ ]:
def plot_f1_and_confusion_matrix(f1_train, f1_valid, cm_train, cm_valid):
    # Plotting F1-Score
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.bar(['Train F1', 'Valid F1'], [f1_train, f1_valid], color=['blue', 'green'])
    plt.title("F1-Score for Training and Validation")

    # Plotting Confusion Matrix
    plt.subplot(1, 2, 2)
    sns.heatmap(cm_valid, annot=True, fmt='d', cmap='Blues')
    plt.title("Confusion Matrix for Validation")
    plt.ylabel('Actual')
    plt.xlabel('Predicted')

    plt.tight_layout()
    plt.show()

In [ ]:
plot_f1_and_confusion_matrix(f1_train_hist[-1], f1_valid_hist[-1], cm_train_hist[-1], cm_valid_hist[-1])

**Importance of ROC and AUC in Evaluation:**

In medical imaging tasks like ours, accuracy isn't always the best performance metric due to potential class imbalances. Instead, we use ROC (Receiver Operating Characteristic) curves and AUC (Area Under the Curve) to provide a more nuanced view of our model's ability to distinguish between classes, ensuring a more reliable assessment of its diagnostic accuracy.

In [ ]:
from sklearn.metrics import roc_curve, auc

def evaluate_and_plot_roc(model, data_loader, device):
    """
    Evaluates the model on the given data loader, calculates the ROC curve and AUC,
    and plots the ROC curve.

    Parameters:
    model (torch.nn.Module): The trained model to evaluate.
    data_loader (torch.utils.data.DataLoader): DataLoader for evaluation data.
    device (torch.device): The device on which the model is.

    Returns:
    None
    """
    model.eval()

    # Hint: Evaluate the model on the data loader, calculate the ROC curve and AUC using sklearn functions,
    # and plot the ROC curve using matplotlib. Make sure to transfer data to the specified device.
    # Your code here [30 score]


In [ ]:
evaluate_and_plot_roc(model, test_loader, device)

# Addressing Class Imbalance

**Explanation**

Class imbalance is a common challenge in medical data analysis. It happens when the number of examples in one class (usually the 'normal' category) is much larger than in another class (often representing a 'disease' condition). This imbalance can lead to models that are unfairly skewed towards the majority class, performing poorly in identifying the crucial, less represented class.


**Why Address Class Imbalance?**

In medical scenarios, the accuracy of detecting rare conditions (the minority class) is as important, if not more so, than identifying common ones. A model biased towards the majority class might overlook these critical minority cases, leading to potential misdiagnoses.


To counter this, one approach is to use a weighted loss function during training. Follow the following hints to implement it.


In [ ]:
# Hint: To address class imbalance, calculate class weights and apply them to the loss function.
# 1. Calculate class distribution: Count the number of samples in each class.
# 2. Calculate imbalance ratio: Compute the ratio of samples in the majority class to the minority class.
# 3. Create class weights: Create a list or tensor containing the class weights based on the imbalance ratio.
#    You may need to use the inverse of the ratio or other strategies to emphasize the minority class.
# 4. Define the loss function: Use nn.BCEWithLogitsLoss and pass the class_weights as the 'weight' argument.

# Your code here [15 score]


In [ ]:
model = create_model(model_name="swin_tiny_patch4_window7_224",
                        pretrained=True,
                        num_classes=1,
                        in_chans=n_channels)
model = model.to(device)

In [ ]:
# Training Loop
n_epochs = 100
loss_train_hist = []
loss_valid_hist = []
acc_train_hist = []
acc_valid_hist = []

for epoch in range(n_epochs):
    torch.cuda.empty_cache()

    model, loss_train, acc_train = train_one_epoch(model, train_loader, criterion, optimizer, epoch)
    loss_valid, acc_valid = validate_one_epoch(model, val_loader, criterion)

    loss_train_hist.append(loss_train)
    loss_valid_hist.append(loss_valid)
    acc_train_hist.append(acc_train)
    acc_valid_hist.append(acc_valid)

    print(f"Epoch: {epoch+1}, Train Loss: {loss_train:.2f}, Train Acc: {acc_train:.2f}, Valid Loss: {loss_valid:.2f}, Valid Acc: {acc_valid:.2f}")

In [ ]:
plot_performance(loss_train_hist, loss_valid_hist, acc_train_hist, acc_valid_hist)

In [ ]:
evaluate_and_plot_roc(model, test_loader, device)

<br><br><br>

In this notebook, you've already learned about addressing imbalanced data using a weighted loss. Now, let's explore an alternative method for tackling class imbalance in machine learning. Please describe and explain this alternative approach.



```
# Your answer here [15 score]
```

